In [1]:
!pip install -U chromedriver-binary
!pip install -U selenium

  Created wheel for chromedriver-binary: filename=chromedriver_binary-97.0.4692.20.0-py3-none-any.whl size=10135190 sha256=00a181a7792e7a284055fd5c186a66dd2b6e09efc2833dea56357b50b9c25707
  Stored in directory: /home/christophearendt/.cache/pip/wheels/d5/d0/ff/4718060215c8c5181c85a13f7efebf5c849e616e14e4235bf8
Successfully built chromedriver-binary
  Attempting uninstall: chromedriver-binary
    Found existing installation: chromedriver-binary 93.0.4577.63.0
    Uninstalling chromedriver-binary-93.0.4577.63.0:
      Successfully uninstalled chromedriver-binary-93.0.4577.63.0
You should consider upgrading via the '/home/christophearendt/.pyenv/versions/3.8.6/envs/lewagon/bin/python3.8 -m pip install --upgrade pip' command.
     |████████████████████████████████| 958 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 356 kB 16.2 MB/s eta 0:00:01
     |████████████████████████████████| 55 kB 1.6 MB/s eta 0:00:011
  Attempting uninstall: selenium
    Found existing installatio

In [15]:
import pandas as pd
from selenium import webdriver
import chromedriver_binary

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

#import toolbox
import toolbox
from toolbox import FileSystem as fs
from toolbox import ImageHandler as ih
from toolbox import FileSystemGUI as gui

ImportError: cannot import name 'FileSystem' from 'toolbox' (/home/christophearendt/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/toolbox/__init__.py)

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--profile-directory=Default')
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-plugins-discovery")
chrome_options.add_argument("--start-maximized")

#### scrapping tools

In [3]:
# needed imports
import os
import time
import re
import requests
import urllib.request
import numpy as np
from tqdm.notebook import tqdm

SEARCH_PAUSE_TIME = 4
SCROLL_PAUSE_TIME = 4
SLEEP_BETWEEN_INTERACTIONS = 1
PATIENCE = 10
sleeps = [1, 0.5, 1.5, 0.7, 0.8, 0.3, 1.1, 1.3]

In [4]:
def scroll_to_end(wd):
    
    last_height = wd.execute_script("return document.documentElement.scrollHeight")
    while True:
        # Scroll down to bottom
        wd.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = wd.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    return wd

In [5]:
def collect_thumbnails(wd, base_image_path):

    # collect all images html elements
    image_links = wd.find_elements_by_class_name('rg_i.Q4LuWd')
    # collect all images sources or vignettes
    src_links = [image_links[i].get_attribute('src')for i in range(len(image_links))]
    data_src_links = [image_links[i].get_attribute('data-src') for i in range(len(image_links))]

    # actual collecting of vignettes for given search
    for i,link in enumerate(tqdm(data_src_links)):
        if link is None:
            link = src_links[i]
        
        dest_path = f"{base_image_path}_{i}.jpeg"
        try:
            dest_path, _ =urllib.request.urlretrieve(link, dest_path)
        except:
            continue # continue to next row
        time.sleep(np.random.choice(sleeps))
    return wd

In [6]:
def wait_for_elements(driver, byType, byValue, maxWait: int):
    """
    This method, takes properties of an element and looks for the element in page.
    if the element is found, it will return the element else None is returned

    Args:
        driver ([webdriver]): [Selenium webdriver]
        byType ([By Type]): [Which property we are using to get element. ex: By.XPATH or BY.Id]
        byValue ([type]): [value of property]
        maxWait (int): [wait for element to be found]

    Returns:
        [webdriverelement]: [identified element]
    """
    try:
        element = WebDriverWait(driver, maxWait).until(
            EC.presence_of_all_elements_located((byType, byValue)))
        return element
    except TimeoutException:
        print("Element not found")
        return None

In [7]:
def collect_links_to_image(wd, path_to_backup_links="", verbose=False):

    image_urls = []
    image_count = 0
    results_start = 0

    # get all image thumbnail results
    thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
    number_results = len(thumbnail_results)
        
    print(f"Found: {number_results} search results.")
    print(f">> Extracting links from {results_start}:{number_results}")   
    for img in  tqdm(thumbnail_results[results_start:number_results]):
        # try to click every thumbnail such that we can get the real image behind it
        try:
            img.click()
            time.sleep(SLEEP_BETWEEN_INTERACTIONS)
        except Exception:
            continue
        
        # extract image urls (avoid images from google cache)  
        actual_images = wait_for_elements(wd, By.CSS_SELECTOR, 'img.n3VNCb', PATIENCE)
        if (None != actual_images):
            for actual_image in actual_images:
                src_ = actual_image.get_attribute('src')
                if src_ is not None and \
                   ('http' in src_ or 'http' in src_) and \
                  'encrypted-tbn0.gstatic.com' not in src_:
                    if verbose is True:
                        print(src_)
                    image_urls.append(src_)
                    if len(path_to_backup_links) > 0:
                        with open(path_to_backup_links, 'a') as f:
                            f.write(f"{src_}\n")

    return image_urls


#### Actual scrapping starts here ... chill out and come back later

In [8]:
print('ENTER YOUR SEARCH CRITERIA:')
quest = input();

ENTER YOUR SEARCH CRITERIA:
arc de triomphe


In [9]:
print('ENTER PATH TO IMAGES SEARCH OUTCOME:')
dataset_directory = gui.gui_fname().decode("utf-8") 

ENTER PATH TO IMAGES SEARCH OUTCOME:


NameError: name 'gui' is not defined

In [ ]:

start = 1
count = start

driver = webdriver.Chrome(options=chrome_options);
driver.get("https://www.google.fr/imghp?hl=fr&ogbl")

# handling accept cookies for search: searching for 'accept' button
ids = driver.find_elements_by_xpath('//*[@id]')
for ii in ids:
    if ii.tag_name == "button" and ii.text == "J'accepte":
        ii.click()
# trigger the actual search for the asana
search = driver.find_element_by_tag_name("input")
search.clear()

# create depositoty for scraping outcome
dataset_path = f"{dataset_directory}/{quest}"
if not fs.exists(dataset_path):
    os.makedirs(dataset_path)  

# create file for storing original links to image
dataset_links_backup_path = f"{dataset_directory}/{quest}/{quest.replace(' ', '_')}.txt"

# trigger the search image engine here 

search.send_keys(quest)
search.submit()

# wait a notch
time.sleep(SEARCH_PAUSE_TIME)

# ----------------------------------------------------------------------------#
# SCROLL TO THE BOTTOM OF THE SEARCH RESULTS till it displays "display more" -# 
# ----------------------------------------------------------------------------#
driver = scroll_to_end(driver)
print('Hit any key to continue')
rr = input();

# ---------------------------------------------------#
# COLLECT IMAGE THUMBNAILS AND BACKUP LINKS TO .TXT -# 
# ---------------------------------------------------#

if not os.path.exists(dataset_links_backup_path):
    list_of_urls = collect_links_to_image(driver, dataset_links_backup_path, verbose=True)
else:
    list_of_urls = [line.strip() for line in open(dataset_links_backup_path, 'r')]
    
# ----------------------------------#
# -    COLLECT IMAGES TO DATASET  - # 
# ----------------------------------#    
print(f">> Downloading dataset images for: {search}")
base_dest_path = dataset_links_backup_path
#collect_thumbnails(driver, base_dest_path)

i = 0
issues = []

for url in tqdm(list_of_urls):
    file_ext_pattern = '\.(?i)(jpe?g|png|gif|tif|cms|bmp|img|webp)'
    match= re.findall(file_ext_pattern,(fs.file_extension(url)))
    if match:
        dest_path = f"{base_dest_path}_{i}.{match[0]}"
        format_is_known = True
    else:
        dest_path = f"{base_dest_path}_{i}"
        format_is_known = False
        
    try:
        req = requests.get(url,timeout=5, headers={"User-Agent": "Chrome/96.0.4664.55"})
        if req.status_code == 200:
            with open(dest_path, 'wb') as f:
                f.write(req.content)
            if format_is_known is False:
                os.rename(dest_path,
                            f"{dest_path}.{ih.get_image_format(dest_path)}")
                
        else: 
            issues.append({f"{req.status_code}":url})

        
    except requests.exceptions.HTTPError as errh:
        issues.append({"Http Error": url})
    except requests.exceptions.ConnectionError as errc:
        issues.append({"Error Connecting": url})
    except requests.exceptions.Timeout as errt:
        issues.append({"Timeout Erro": url})
    except requests.exceptions.RequestException as err:
        issues.append({"RequestException": url})  
    except:
        issues.append({"Some other exception...": url})
        
    time.sleep(np.random.choice(sleeps))
    i+=1
if len(issues) > 0:
    print(f">> Reporting issues for scrapping: {quest}")
    for d in issues:
        print(f" --- type: {list(d.keys())[0]} - link: {list(d.values())[0]}") 
# try to end gracefully...
time.sleep(SCROLL_PAUSE_TIME)
driver.quit()
count += 1